In [33]:
from os.path import dirname, join

import pandas as pd

from bokeh.io import curdoc, output_notebook
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import PreText, Select
from bokeh.plotting import figure, show
from bokeh.client import push_session


#%cd /home/brett/Documents/JupyterNotebooks/AgResearch/OrganicGraph

try:
    from functools import lru_cache
except ImportError:
    # Python 2 does stdlib does not have lru_cache so let's just
    # create a dummy decorator to avoid crashing
    print ("WARNING: Cache for this example is available on Python 3 only.")
    def lru_cache():
        def dec(f):
            def _(*args, **kws):
                return f(*args, **kws)
            return _
        return dec


DEFAULT_TICKERS = ['Artichoke (SF only); Globe; cartons, 24s; San Fran',
 'Cabbage; Round green type; 40 lb cartons, medium; Atlanta',
 'Cabbage; Round green type; 45 lb cartons, medium; Atlanta',
 'Cabbage; Round green type; 45 lb cartons, 18-24s; San Fran',
 'Carrots; Not specified; 25 lb sacks loose; Atlanta',
 'Carrots; Topped; 25 lb sacks loose; San Fran',
 'Cauliflower; White; cartons, film wrapped, 12s; Atlanta',
 'Cauliflower; White; cartons, film wrapped, 12s; San Fran',
 'Greens; Swiss Chard, mixed colors; cartons bunched, 24s; Atlanta',
 'Greens; Swiss Chard, mixed colors; cartons bunched, 24s; San Fran',
 'Lettuce; Green leaf; cartons, 24s; Atlanta',
 'Lettuce; Green leaf; cartons, 24s; San Fran',
 'Lettuce; Romaine; cartons, 24s; Atlanta',
 'Lettuce; Romaine; cartons, 24s; San Fran',
 'Mesculin Mix; NaN; 3 lb cartons; Atlanta',
 'Mesculin Mix; NaN; 3 lb cartons; San Fran',
 'Onions, Dry; Yellow, Hybrid; All packages - no 40 lb cartons, JBO; Atlanta',
 'Onions, Dry; Yellow, Hybrid; 40 lb cartons, JBO; Atlanta',
 'Onions, Dry; Yellow, Hybrid; 40 lb cartons, JBO; San Fran',
 'Spinach; Flat; cartons, bunched, 24s; Atlanta',
 'Spinach; Flat; cartons, bunched, 24s; San Fran',
 'Potatoes; Russet; 50 lb cartons, US. No.1; Atlanta',
 'Potatoes; Russet; 50 lb cartons, US. No.1; San Fran',
 'Tomatoes, Cherry; no Pear type; no Flats 12 1/2 pt cups with lids; Atlanta',
 'Tomatoes, Cherry; no Pear type; Flats 12 1-pt cups with lids; San Fran',
 'Tomatoes, Cherry; Pear type; Flats 12 1/2 pt cups with lids; San Fran',
 'Sweet Potatoes; Orange types; 40 lb cartons, U.S. No 1; Atlanta',
 'Sweet Potatoes; Orange types; 40 lb cartons, U.S. No 1; San Fran']
    
def nix(val, lst):
    return [x for x in lst if x!= val]

@lru_cache()
def load_ticker(ticker):
    Ags = pd.read_pickle('Ags.pkl')
    data = Ags[Ags.Uniquely == ticker]
    return pd.DataFrame({'Conventional' : data.Price_Conv, 'Organic' : data.Price_Org, 
                         'OrganicPremium': data.Premium})

@lru_cache()
def get_data(v1):
    data = load_ticker(v1)  
    return data

#set up widget
ticker = Select(title="Veggies:", value='Lettuce; Green leaf; cartons, 24s; Atlanta', 
                options=nix('Lettuce; Green leaf; cartons, 24s; Atlanta',DEFAULT_TICKERS))
stats = PreText(text='', width=500)

#set up Plots
source = ColumnDataSource(
    data=dict(MonthYearForm=[],Organic=[],Conventional=[], OrganicPremium=[]))

source_static = ColumnDataSource(
    data=dict(MonthYearForm=[],Organic=[],Conventional=[], OrganicPremium=[]))

tools = 'pan,wheel_zoom,xbox_select,reset'

corr = figure(plot_width=350, plot_height=350,
              tools='pan,wheel_zoom,box_select,reset')

corr.circle('Organic', 'Conventional', size=4, source=source,
            selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)


ts1 = figure(plot_width=900, plot_height=200, tools=tools, x_axis_type='datetime', active_drag="xbox_select")
ts1.line('MonthYearForm', 'Organic', source=source_static)
ts1.circle('MonthYearForm', 'Organic', size=1, source=source, color=None, selection_color="orange")


ts2 = figure(plot_width=900, plot_height=200, tools=tools, x_axis_type='datetime', active_drag="xbox_select")
ts2.x_range = ts1.x_range
ts2.line('MonthYearForm', 'Conventional', source=source_static)
ts2.circle('MonthYearForm', 'Conventional', size=1, source=source, color=None, selection_color="orange")


#set up callbacks
def ticker_change(attrname, old, new):
    ticker.options = nix(new, DEFAULT_TICKERS)
    update()

def update(selected=None):
    v1 = ticker.value
    data = get_data(v1)
    source.data=source.from_df(data[['Conventional', 'Organic', 'OrganicPremium']])
    source_static.data = source.data
    
    update_stats(data, v1)
    
    corr.title.text = v1

def update_stats(data, v1):
    stats.text = str(data[['Conventional', 'Organic', 'OrganicPremium']].describe())    
    
    
ticker.on_change('value',ticker_change)

def selection_change(attrname, old, new):
    v1 = ticker.value
    data=get_data(v1)
    selected = source.selected['1d']['indices']
    if selected:
        data = data.iloc[selected, :]
    update_stats(data, v1)

source.on_change('selected',selection_change)

# set up layout
widgets = column(ticker, stats)
main_row = row(corr, widgets)
series = column(ts1, ts2)
layout = column(main_row, series)

update()

curdoc().add_root(layout)

curdoc().title='Veggies'